## preliminaries

In [1]:
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.algorithms.rapidart as ra      # artifact detection
import nipype.algorithms.modelgen as model   # model specification
import os                                    # system functions


In [2]:
spm_standalone='/homes_unix/hirsch/historique_fli_iam/essai_spm_stand_alone/spm12/run_spm12.sh'
mcr='/homes_unix/hirsch/historique_fli_iam/essai_spm_stand_alone/mcr2016/v91'

In [3]:
from nipype.interfaces import spm

matlab_cmd = ' '.join([spm_standalone,mcr,'batch','script'])
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

## 1 - we decode the xml file

In [4]:
flibasedir = '/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01'

In [5]:
# we get the xml file in this directory
def getXmlFile(flibasedir):
    import os
    dirs = os.listdir( flibasedir )
    #print dirs

    import re
    xmlmatch = r'(.*)xml'
    for dir in dirs:
        xmlfound=re.search(xmlmatch,dir,flags=0)
        if xmlfound:
            #print xmlfound.group()
            return flibasedir + '/' + xmlfound.group()

In [6]:
xmlfile = getXmlFile(flibasedir)

print xmlfile

/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/xml_example.xml


In [7]:
# the we decode the xmlfile
import xml.etree.ElementTree as ET
tree = ET.parse(xmlfile)
root = tree.getroot()

In [8]:
print root.tag

def getEpiInfos(xmlfile):
    result = {}
    try:
        import xml.etree.ElementTree as ET
        tree = ET.parse(xmlfile)
        root = tree.getroot()
        # epibold directory
        for epibold in root.iter('EPIBOLD'):
            for child in epibold:
                print child.text
                if child.tag == 'file':
                    epidir = child.text
                    result['epidir'] = epidir
                    print epidir
                    
                # parameters
                if child.tag == 'parameters':
                    print "parameters"
                    
                    for child2 in child:
                                                     
                        # TR
                        if child2.tag == 'TR':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    tr = child3.text
                                    print tr
                                    result['TR'] = tr
                    
                        # dynamics
                        if child2.tag == 'dynamics':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    dynamics = child3.text
                                    result['dynamics'] = dynamics
                                    
                        # sliceTimingVector
                        if child2.tag == 'sliceTimingVector':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    sliceTimingVector = child3.text
                                    result['sliceTimingVector'] = sliceTimingVector
                                    
                        # nb_slices
                        if child2.tag == 'nb_slices':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    nb_slices = child3.text
                                    result['nb_slices'] = nb_slices
                
        
    except:
        print 'exception'
    return result


RS_analysis


In [9]:
epiResults = getEpiInfos(xmlfile)
print epiResults

/EPIBOLD/
/EPIBOLD/
bold-dataset

     
parameters
2
{'sliceTimingVector': '1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 2 4 6 8 10 12 14 16 18 20 22 24 26 28 30', 'dynamics': '240', 'nb_slices': '31', 'TR': '2', 'epidir': '/EPIBOLD/'}


In [10]:
def getT1file(xmlfile):
    try:
        import xml.etree.ElementTree as ET
        tree = ET.parse(xmlfile)
        root = tree.getroot()
        # T1 directory and file
        for t1 in root.iter('T1'):
            for child in t1:
                #print child.text
                if child.tag == 'file':
                    t1 = child.text
                    t1 = flibasedir + t1
                    # print  t1
    except:
        print 'exception'
    return t1
    
    

In [11]:
t1 = getT1file(xmlfile)
print t1

/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/T1/t0009_t1_s03.nii


# Pileline declaration

In [12]:
preproc = pe.Workflow(name='preproc')

## first node data grabbing by selcet files 

In [13]:
epidir = getEpiInfos(xmlfile)['epidir']
epidir = flibasedir + epidir

from nipype import SelectFiles, Node
templates = dict(T1=t1,
                 epi= epidir + "/" + "*.nii")

filesource = Node(SelectFiles(templates), "filesource")
filesource.inputs.subject_id = "subj1"
filesource.outputs.get()


/EPIBOLD/
/EPIBOLD/
bold-dataset

     
parameters
2


{'T1': <undefined>, 'epi': <undefined>}

## segment T1

In [14]:
segment = pe.Node(interface=spm.NewSegment(), name='segment')

#seg.inputs.channel_files = 't0009_t1_s03.nii'

segment.inputs.channel_info = (0.0001, 60, (True, True))
tissue1 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 1), 2, (True,True), (False, False))
tissue2 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 2), 2, (True,True), (False, False))
tissue3 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 3), 2, (True,False), (False, False))
tissue4 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 4), 2, (False,False), (False, False))
tissue5 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 5), 2, (False,False), (False, False))
segment.inputs.tissues = [tissue1, tissue2, tissue3, tissue4, tissue5]


# connection du pipeline

In [ ]:
preproc.connect(filesource,"T1" ,segment, 'channel_files')

## datasink

In [15]:
datasink = pe.Node(nio.DataSink(), name='datasink')
datasink.inputs.base_directory = '/homes_unix/hirsch/_pypipe/datadir/data_results'

In [16]:
preproc.connect([(filesource, segment, [('T1', 'channel_files')]),
                      (segment, datasink, [('normalized_class_images', 'normalized'),
                                              ('bias_corrected_images', 'bias_corrected')
                                              ])
                      ])

In [17]:
preproc.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node filesource in dir: /tmp/tmp6Xftdm/preproc/filesource
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node segment in dir: /tmp/tmp8j7BO2/preproc/segment
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node datasink in dir: /tmp/tmpOU30e5/preproc/datasink
INFO:workflow:Runtime memory and threads stats unavailable


In [ ]:
#advanced way to connect multiple nodes
workflowname.connect([(nodename1, nodename2, [('output_node1', 'input_node2')]),
                      (nodename1, nodename3, [('output_node1', 'input1_node3')]),
                      (nodename2, nodename3, [('output1_node2', 'input1_node3'),
                                              ('output2_node2', 'input2_node3')
                                              ])
                      ])

In [ ]:
datasink = pe.Node(nio.DataSink(), name='datasink')
datasink.inputs.base_directory = '/homes_unix/hirsch/_pypipe/datadir/data_results'
preproc.connect(segment,  'normalized_class_images', datasink, 'structural')
preproc.connect(segment,  'bias_corrected_images' , datasink, 'structural.@par')


In [ ]:
preproc.run()

In [ ]:
spm.NewSegment.help()

In [ ]:
spm.NewSegment.input_spec()

In [ ]:
spm.NewSegment.output_spec()

In [ ]:
/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM

In [ ]:
#Where should the output data be stored at?
sink = nipype.DataSink()
sink.inputs.base_directory = experiment_dir + '/output_folder'

In [ ]:
%cd /homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/EPIBOLD
%pwd